# `Setup`

In [1]:
import subprocess

try:
    subprocess.run(["docker", "inspect", "--format", "'{{.State.Running}}'", "redis"], check=True)
    print("Redis is already running.")
except subprocess.CalledProcessError:
    print("Starting Redis...")
    subprocess.run(["docker", "run", "--name", "redis", "-p", "6379:6379", "-d", "redis/redis-stack:latest"])
    print("Redis started.")

'true'
Redis is already running.


# `Dict input, message output`

In [2]:
from typing import Optional

from langchain_community.chat_message_histories import RedisChatMessageHistory
from langchain_community.chat_models import ChatOllama
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory

In [3]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You're an assistant who's good at {ability}"),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}"),
])

chain = prompt | ChatOllama(model="mistral")

In [ ]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: RedisChatMessageHistory(session_id),
    input_messages_key="question",
    history_messages_key="history",
)

In [ ]:
chain_with_history.invoke(
    {"ability": "math", "question": "What does cosine mean?"},
    config={"configurable": {"session_id": "foobar"}},
)

In [ ]:
chain_with_history.invoke(
    {"ability": "math", "question": "What's its inverse"},
    config={"configurable": {"session_id": "foobar"}},
)

## `Messages input, dict output`

In [ ]:
from langchain_core.messages import HumanMessage
from langchain_core.runnables import RunnableParallel

chain = RunnableParallel({"output_message": ChatOllama(model="mistral")})
chain_with_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: RedisChatMessageHistory(session_id),
    output_messages_key="output_message",
)

chain_with_history.invoke(
    [HumanMessage(content="What did Simone de Beauvoir believe about free will")],
    config={"configurable": {"session_id": "baz"}},
)

In [ ]:
chain_with_history.invoke(
    [HumanMessage(content="How did this compare to Sartre")],
    config={"configurable": {"session_id": "baz"}},
)

## `With itemgetter`

In [ ]:
from operator import itemgetter

RunnableWithMessageHistory(
    ChatOllama(model="mistral"),
    lambda session_id: RedisChatMessageHistory(session_id),
)

RunnableWithMessageHistory(
    itemgetter("input_messages") | ChatOllama(model="mistral"),
    lambda session_id: RedisChatMessageHistory(session_id),
    input_messages_key="input_messages",
)

# `Tear down`

In [ ]:
try:
    subprocess.run(["docker", "inspect", "--format", "'{{.State.Running}}'", "redis"], check=True)
    subprocess.run(["docker", "stop", "redis"])
    subprocess.run(["docker", "rm", "redis"])
    print("Redis has been stopped and removed.")

except subprocess.CalledProcessError:
    print("Redis is not currently running.")